# MCMC2.0: Iterators, Generators, and Channels

Iterators and generators are two important tools in the modern functional programming and Julia implements new "channels." I will show a typical implementation of them for MCMC.

## Iterators

Iterators are often used to save memories. If we wish to do something to each element of a large list, we do not have to store all the elements between the processes. Iterators return each element one by one, so we can do opearations one by one and each time we can discard old data to save memories. Actually, we already used iterators in usual `for` loops.

In [1]:
iter = 1 : 2

1:2

This is already iterator counting from 1 to 2.

In [2]:
element, state = iterate(iter)

(1, 1)

This is the initial state of the iterator `1:2`.

In [3]:
element, state = iterate(iter, state)

(2, 2)

These are the next item and the next state.

In [4]:
result = iterate(iter, state)
result == nothing

true

If there is nothing to iterate, they return nothing.

In [5]:
iter = 1 : 5
for i in iter
    println(i)
end

1
2
3
4
5


This is the same as follow:

In [6]:
iter_result = iterate(iter)
while iter_result !== nothing
    (i, state) = iter_result
    println(i)
    iter_result = iterate(iter, state)
end

1
2
3
4
5


If we use an iterator instead of a list, we do not need to store the whole list anymore.

In [7]:
list = collect(iter)
for i in 1 : length(list)
    println(list[i] ^ 2)
end

1
4
9
16
25


This is meaningless and should be rewritten as:

In [8]:
for i in iter
    println(i ^ 2)
end

1
4
9
16
25


In this way, we can (trivially) avoid allocating a memory for the list. Or this works if the memory does not matter:

In [9]:
for i in list
    println(i ^ 2)
end

1
4
9
16
25


## IterTools

~ under construction ~

In [10]:
using IterTools
using Distributions
"""
Curried form of a Gibbs sampler
"""
function Gibbs1(a::Float64, b::Float64, c::Float64)::Function
    xy::Tuple{Float64, Float64} -> begin
        x = rand(Normal(b / a * xy[2], 1 / sqrt(a)))
        x, rand(Normal(b / c * x, 1 / sqrt(c)))
    end
end
samples = iterated(Gibbs1(1.0, 0.8, 1.0), (0.0, 0.0))
collect(Iterators.take(samples, 10))

10-element Array{Tuple{Float64,Float64},1}:
 (0.0, 0.0)                                 
 (0.4165394329958923, -1.057282647136194)   
 (0.22859089945139766, -0.3018351454637973) 
 (0.17980671494992087, -0.43518595303679486)
 (-0.4076420880403845, 1.2630831431643859)  
 (2.5276833491548873, 3.7216787527132724)   
 (1.5704146035824729, 2.338297583308654)    
 (0.8738249284502068, -0.46577656225248754) 
 (0.4146971404012101, -1.3697733203771485)  
 (-1.0516811921859786, -0.691684460268625)  

## Generator 1 (generator expression)

An iterator is most useful when combined with a generator. We generally call something creating an iterator a generator. Generators in general include a generator expression, a generator function, etc. Julia supports a generator expression as follows:

In [11]:
gene = (x^2 for x in 1 : 5)

Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}(getfield(Main, Symbol("##5#6"))(), 1:5)

This is very similar to a list comprehension:

In [12]:
list = [x^2 for x in 1 : 5]

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

The list comprehension will allocate a memory for the whole list. The generator will save your memory.

In [13]:
for i in gene
    println(i)
end

1
4
9
16
25


## Generator 2 (generator function = resumable function)

An easier, simpler, and most useful way to create an iterator/generator is a generator function, and Python and most modern language support this type of functions. Julia does not natively support Python-type generator functions, but simply you can call ResumableFunctions.jl. This is the easiest way to reproduce the behavior of generator functions.

In [14]:
using ResumableFunctions
@resumable function Ising()::Vector{Int64}
    N = 8
    σ = ones(Int64, N)
    β = 1.0
    while true
        for i in 1 : 1000
            j = rand(1 : N)
            ΔβE = 2β * σ[j] * (σ[mod1(j + 1, N)] + σ[mod1(j - 1, N)])
            -ΔβE > log(rand()) && (σ[j] = -σ[j])
        end
        @yield σ
    end
end

Ising (generic function with 1 method)

With `@resumable` macro, you can define a resumable function. This function does not have `return`, but yield what you want at `@yield` points. This function directely returns an iterater, i.e. a function iterate(iter) is already defined when we call `iter = Ising()`. Then, this iterater gives you the "yield" repeatedly each time you reached `@yield` point.

In [15]:
iter = Ising()
@show iter();

iter() = [-1, -1, -1, -1, 1, 1, -1, -1]


In [16]:
for i in 1 : 10
    println(iter())
end

[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, 1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]


ResummableFunctions are very fast and memory-efficient because the macro `@resummable` will rewrite a function into a normal iterator directly before the JIT compilation.

## Channels

Instead of a generator function, Julia natively supports Channels for parallel computing. Especially, `Channel` is a stronger and more general tool to create an iterator from the function. This is a generalized concept of a generator function, but in most cases ResumableFunctions.jl is faster and memory-efficient. However, if you need a postprocess based on lazy evaluation, Channels work better from my experience.

In [17]:
"""
Channel form of a Gibbs sampler
"""
function Gibbs2(a::Float64, b::Float64, c::Float64)::Channel
    Channel(ctype = Tuple{Float64,Float64}) do channel::Channel{Tuple{Float64,Float64}}
        N = 10
        x = 0.0
        y = 0.0
        put!(channel, (x, y))
        for i in 1 : N
            x = rand(Normal(b / a * y, 1 / sqrt(a)))
            y = rand(Normal(b / c * x, 1 / sqrt(c)))
            put!(channel, (x, y))
        end
    end
end

Gibbs2

In [18]:
for z in Gibbs2(1.0, 0.8, 1.0)
    println(z)
end

(0.0, 0.0)
(1.156203257829569, 0.29213018663385193)
(-0.5625462991060775, -1.1738574983258179)
(-2.356207514241186, -0.08951024214860115)
(-0.8501163072123769, -0.9638549789201147)
(0.0174543992722328, 1.43460518355639)
(0.8612748442533313, 0.3113904231513061)
(0.027712300631905246, -0.9905624836046465)
(-0.9635861998887635, 0.0011901960934292966)
(0.19200212168436515, 1.72281276084399)
(0.11683636760363614, 1.2185398604055206)


This code is acceptable if the number of tasks is just one.

~ under construction ~

In [19]:
"""
Channel form of the first part of a Gibbs sampler
"""
function xupdate(a::Float64, b::Float64, c::Float64, input::Channel{Tuple{Float64,Float64}}, output::Channel{Tuple{Float64,Float64}})
    N = 10
    x = 0.0
    y = 0.0
    for i in 1 : N
        (x, y) = take!(input)
        x = rand(Normal(b / a * y, 1 / sqrt(a)))
        put!(output, (x, y))
    end
    close(output)
end
"""
Channel form of the second part of a Gibbs sampler
"""
function yupdate(a::Float64, b::Float64, c::Float64, input::Channel{Tuple{Float64,Float64}}, output::Channel{Tuple{Float64,Float64}})
    N = 10
    x = 0.0
    y = 0.0
    for i in 1 : N
        (x, y) = take!(input)
        y = rand(Normal(b / c * x, 1 / sqrt(c)))
        println((x, y))
        put!(output, (x, y))
    end
    close(output)
end
ch1 = Channel{Tuple{Float64,Float64}}(32)
ch2 = Channel{Tuple{Float64,Float64}}(32)
@async xupdate(1.0, 0.8, 1.0, ch1, ch2)
@async yupdate(1.0, 0.8, 1.0, ch2, ch1)
put!(ch1, (0.0, 0.0));

(-0.3186929194386921, -1.6702196109784313)
(-1.011238401132959, -1.1137331843041753)
(-0.06347304067600812, 0.32722712634202045)
(-1.1168758720121474, 0.21278196747454658)
(1.061581021191395, 1.5963694962286303)
(2.5577495903790837, 1.1615286840188892)
(1.011293093958137, 0.9537905467374062)
(0.490973222963105, 1.2569308942017625)
(2.079211933280665, -0.017584658184292667)
(0.8642330508869412, 0.8209648265182741)


The difference bewteen Channels and ResumableFunctions lies in how they compile a code. Check: https://white.ucc.asn.au/2017/11/18/Lazy-Sequences-in-Julia.html

## RemoteChannels

It will be discussed in MCMC4.0.

## Lazy

Lazy.jl is sometimes useful.

In [20]:
import Lazy.takewhile
takewhile(x -> x < 3, 1 : 5)

(1 2)

However, the coding style of Lazy.jl is different from ordinarly Julia, so I won't use this package unless it is necessary.

**Exercise**: implement takewhile and dropwhile for iterators. These functions are sometimes useful for MCMC.